# Linear regression
Classical linear regression is one of the machine learning alghoritms that aims to predict the value of an dependent variable Y (based on one or more independent variables X). This relationship is formed using the last squares method, which minimizes the sum of squares of the deviation errors between the true value (existing in the dataset) and the predicted value by plotting a straight line fit to the dataset. Least squares method estimators are the best estimators from the class of linear loaded estimators - their values can be used to estimate future values of a given trait (e.g., house price) based on the explanatory characteristics provided (e.g., murder rate or number of rooms in a house).

In [45]:
lin.reg <- function(Y, X, data){
   Y <- as.matrix(data[Y])
    
   stopifnot(is.numeric(Y))
   stopifnot(length(X) > 0 || as.character(X))
    
   X <- as.matrix(data[X])
   X2 <- cbind(Intercept = 1, X)
    
   beta <- as.vector(solve(t(X2) %*% X2) %*% t(X2) %*% Y)
    
   X2.nrow <- nrow(X2)
   X2.ncol <- ncol(X2)
    
   Y2 <- matrix(, nrow = X2.nrow, ncol = X2.ncol-1)
    
   for (i in 1:X2.ncol-1) {
      Y2[,i] <- beta[i+1] * X[,i]
   }
   Y2 <- rowSums(Y2) + beta[1]
   eps <- Y - Y2
   S2e <- sum(eps^2) / (X2.nrow - X2.ncol)
   S2b <- S2e * solve(t(X2) %*% X2)
   Sb <- sqrt(diag(S2b))
    
   t.stat <- beta / Sb
   T.p.val <- 2 * pt(abs(t.stat), X2.nrow-X2.ncol, lower.tail = FALSE)
    
   R2 <- 1 - sum(eps^2)/sum((Y - mean(Y))^2)
    
   n <- length(Y)
   p <- ncol(X)
   R2.adj <- 1 - (1 - R2) * (n - 1) / (n - p - 1)
    
   F.stat = R2 / (1 - R2) * (X2.nrow - X2.ncol) / (X2.ncol - 1)
   F.p.val <- pf(F.stat, (X2.ncol - 1), (X2.nrow - X2.ncol), lower.tail = FALSE)
    
   results <- list(Summary = cbind(Estimate = beta, t.value = t.stat, T.p.value = T.p.val),
                    R2 = cbind(R2, R2.adj), F = cbind(F.value = F.stat, F.p.value = F.p.val))
   results[] <- lapply(results, round, 4) 
         
}

## Case: BostonHousing dataset

In [46]:
library(mlbench)

In [47]:
data(BostonHousing)

str(BostonHousing)
head(BostonHousing)
summary(BostonHousing)

'data.frame':	506 obs. of  14 variables:
 $ crim   : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ zn     : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ indus  : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ chas   : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ nox    : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ rm     : num  6.58 6.42 7.18 7 7.15 ...
 $ age    : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ dis    : num  4.09 4.97 4.97 6.06 6.06 ...
 $ rad    : num  1 2 2 3 3 3 5 5 5 5 ...
 $ tax    : num  296 242 242 222 222 222 311 311 311 311 ...
 $ ptratio: num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ b      : num  397 397 393 395 397 ...
 $ lstat  : num  4.98 9.14 4.03 2.94 5.33 ...
 $ medv   : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
6,0.02985,0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


      crim                zn             indus       chas         nox        
 Min.   : 0.00632   Min.   :  0.00   Min.   : 0.46   0:471   Min.   :0.3850  
 1st Qu.: 0.08205   1st Qu.:  0.00   1st Qu.: 5.19   1: 35   1st Qu.:0.4490  
 Median : 0.25651   Median :  0.00   Median : 9.69           Median :0.5380  
 Mean   : 3.61352   Mean   : 11.36   Mean   :11.14           Mean   :0.5547  
 3rd Qu.: 3.67708   3rd Qu.: 12.50   3rd Qu.:18.10           3rd Qu.:0.6240  
 Max.   :88.97620   Max.   :100.00   Max.   :27.74           Max.   :0.8710  
       rm             age              dis              rad        
 Min.   :3.561   Min.   :  2.90   Min.   : 1.130   Min.   : 1.000  
 1st Qu.:5.886   1st Qu.: 45.02   1st Qu.: 2.100   1st Qu.: 4.000  
 Median :6.208   Median : 77.50   Median : 3.207   Median : 5.000  
 Mean   :6.285   Mean   : 68.57   Mean   : 3.795   Mean   : 9.549  
 3rd Qu.:6.623   3rd Qu.: 94.08   3rd Qu.: 5.188   3rd Qu.:24.000  
 Max.   :8.780   Max.   :100.00   Max.   :12.1

In [48]:
lm_fit_mine <- lin.reg(Y = "tax", X = c("crim", "zn", "indus", "nox", "rm", "age",
                                        "dis", "rad", "ptratio", "b", "lstat", "medv"),
                       data = BostonHousing)

lm_fit_mine

,Estimate,t.value,T.p.value
Intercept,212.9354,3.3747,0.0008
crim,-0.2667,-0.6749,0.5000
zn,0.8888,5.5414,0.0000
indus,6.9524,10.5688,0.0000
nox,37.5210,0.8096,0.4186
rm,-1.1956,-0.2223,0.8241
age,0.0879,0.5600,0.5757
dis,-1.7252,-0.6898,0.4906
rad,14.1389,28.7632,0.0000
ptratio,1.0859,0.6634,0.5074


In [49]:
lm_fit_R <- lm(tax ~ . - chas, data = BostonHousing)

summary(lm_fit)


Call:
lm(formula = tax ~ . - chas, data = BostonHousing)

Residuals:
     Min       1Q   Median       3Q      Max 
-214.361  -21.716   -4.506   13.241  262.865 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 212.935376  63.097963   3.375 0.000797 ***
crim         -0.266723   0.395195  -0.675 0.500046    
zn            0.888804   0.160395   5.541 4.89e-08 ***
indus         6.952432   0.657825  10.569  < 2e-16 ***
nox          37.520956  46.346713   0.810 0.418577    
rm           -1.195643   5.377484  -0.222 0.824139    
age           0.087930   0.157021   0.560 0.575740    
dis          -1.725215   2.500886  -0.690 0.490618    
rad          14.138909   0.491563  28.763  < 2e-16 ***
ptratio       1.085922   1.636963   0.663 0.507400    
b            -0.004533   0.032351  -0.140 0.888620    
lstat        -1.131543   0.664282  -1.703 0.089122 .  
medv         -1.915798   0.524456  -3.653 0.000287 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 